In [3]:
import requests
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [28]:
import MetaTrader5 as mt5
mt5.initialize()
# Replace following with your MT5 Account Login
account=51434456 # 
password="9UpBvVzc"
server = 'ICMarkets-Demo'

def get_rates(pair1, timeframe, x):
    pair1 = pd.DataFrame(mt5.copy_rates_from_pos(pair1, timeframe, 0, x))
    pair1['time'] = pd.to_datetime(pair1['time'], unit = 's')
    return pair1[['time','open', 'high', 'low', 'close']].set_index('time')


In [29]:
base_url = 'https://stats.bis.org/api/v1'

In [30]:
import json

In [58]:
import requests

# Define the base URL for the BIS Stats API
base_url = 'https://stats.bis.org/api/v1'

# Define the endpoint parameters
flow = 'BIS,WS_EER_D,1.0'  # Example: Version 1.0 of the WS_EER_M domain, maintained by the BIS
key = 'D.N.N.US'  # Example: Monthly nominal rate in the broad basket for Switzerland
start_period = '2015'  # Example: Start year 2000
end_period = '2023'  # Example: End year 2020
detail = 'full'  # Example: All data and documentation

# Construct the endpoint URL
endpoint_url = f'{base_url}/data/{flow}/{key}/all'

# Define the query parameters
query_params = {
    'startPeriod': start_period,
    'endPeriod': end_period,
    'detail': detail
}

# Make the GET request
response = requests.get(endpoint_url, params=query_params)

# Check for a successful response
if response.status_code == 200:
    # The response data format is XML, so we'll just print the text of the response
    print(response.text)
else:
    print(f'Failed to retrieve data: {response.status_code}')

<?xml version="1.0" encoding="UTF-8"?><message:StructureSpecificData xmlns:ss="http://www.sdmx.org/resources/sdmxml/schemas/v2_1/data/structurespecific" xmlns:footer="http://www.sdmx.org/resources/sdmxml/schemas/v2_1/message/footer" xmlns:ns1="urn:sdmx:org.sdmx.infomodel.datastructure.Dataflow=BIS:WS_EER_D(1.0):ObsLevelDim:TIME_PERIOD" xmlns:message="http://www.sdmx.org/resources/sdmxml/schemas/v2_1/message" xmlns:common="http://www.sdmx.org/resources/sdmxml/schemas/v2_1/common" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"><message:Header><message:ID>IDREFa26588b1-edfa-4bb9-b086-c306abb8a329</message:ID><message:Test>false</message:Test><message:Prepared>2023-12-01T10:09:09Z</message:Prepared><message:Sender id="UNKNOWN"></message:Sender><message:Receiver id="ANONYMOUS"></message:Receiver><message:Structure structureID="BIS_WS_EER_D_1_0" namespace="urn:sdmx:org.sdmx.infomodel.datastructure.Dataflow=BIS:WS_EER_D(1.0):ObsLevelDim:TIME_PERIOD" dimensionAtObservation="TIME_PERIOD"

### AUDUSD

In [32]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd

# Define the base URL for the BIS Stats API
base_url = 'https://stats.bis.org/api/v1'

# Define the endpoint parameters
flow = 'BIS,WS_EER_D,1.0'  # Example: Version 1.0 of the WS_EER_M domain, maintained by the BIS
key = 'D.N.N.AU'  #
start_period = '2000'  # Example: Start year 2000
end_period = '2023'  # Example: End year 2020
detail = 'full'  # Example: All data and documentation

# Construct the endpoint URL
endpoint_url = f'{base_url}/data/{flow}/{key}/all'

# Define the query parameters
query_params = {
    'startPeriod': start_period,
    'endPeriod': end_period,
    'detail': detail
}

# Make the GET request
response = requests.get(endpoint_url, params=query_params)

# Check for a successful response
if response.status_code == 200:
    # Assign the text of the response to xml_data
    xml_data = response.text
    
    # Parse the XML data
    root = ET.fromstring(xml_data)
    
    # Initialize empty lists to store the data
    time_periods = []
    obs_values = []
    
    # Iterate through the XML and extract the desired information
    for obs in root.findall(".//Obs"):
        time_period = obs.get('TIME_PERIOD')
        obs_value = obs.get('OBS_VALUE')
        time_periods.append(time_period)
        obs_values.append(obs_value)
    
    # Create a DataFrame
    df = pd.DataFrame({
        'Time_Period': time_periods,
        'OBS_Value': obs_values
    })
    df = df.dropna()
    # Display the DataFrame
    print(df)
else:
    print(f'Failed to retrieve data: {response.status_code}')

     Time_Period OBS_Value
0     2000-01-01       NaN
1     2000-01-02       NaN
2     2000-01-03     99.49
3     2000-01-04     98.92
4     2000-01-05     99.02
...          ...       ...
8728  2023-11-24    102.54
8729  2023-11-25       NaN
8730  2023-11-26       NaN
8731  2023-11-27    102.82
8732  2023-11-28    102.84

[8733 rows x 2 columns]


In [47]:

df['OBS_Value'] = df['OBS_Value'].replace('NaN', np.nan)
# Drop rows with NaN values
df.dropna(subset=['OBS_Value'], inplace=True)
df['OBS_Value'] = df['OBS_Value'].astype(float)
df['Time_Period'] = pd.to_datetime(df['Time_Period'])
df = df.set_index('Time_Period')
df.tail(10)

,OBS_Value
Time_Period,
2023-11-15,102.21
2023-11-16,101.69
2023-11-17,101.76
2023-11-20,102.18
2023-11-21,102.24
2023-11-22,102.43
2023-11-23,102.37
2023-11-24,102.54
2023-11-27,102.82


In [48]:
AUDUSD = get_rates('AUDUSD.a', mt5.TIMEFRAME_D1, 500)

In [49]:
AUDUSD = get_rates('AUDUSD.a', mt5.TIMEFRAME_D1, 500)
combined_df = pd.concat([df[-len(AUDUSD):], AUDUSD['close']], join = 'outer', axis = 1)

combined_df = combined_df.dropna()
combined_df

,OBS_Value,close
2023-01-02,104.73,0.67977
2023-01-03,103.95,0.67270
2023-01-04,105.88,0.68380
2023-01-05,105.70,0.67521
2023-01-06,104.91,0.68751
...,...,...
2023-11-22,102.43,0.65427
2023-11-23,102.37,0.65575
2023-11-24,102.54,0.65801
2023-11-27,102.82,0.66066


In [52]:
from statsmodels.tsa.stattools import grangercausalitytests
grangercausalitytests(combined_df[['OBS_Value', 'close']], 10)


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=20.3788 , p=0.0000  , df_denom=233, df_num=1
ssr based chi2 test:   chi2=20.6412 , p=0.0000  , df=1
likelihood ratio test: chi2=19.7879 , p=0.0000  , df=1
parameter F test:         F=20.3788 , p=0.0000  , df_denom=233, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=41.1987 , p=0.0000  , df_denom=230, df_num=2
ssr based chi2 test:   chi2=84.1887 , p=0.0000  , df=2
likelihood ratio test: chi2=71.9563 , p=0.0000  , df=2
parameter F test:         F=41.1987 , p=0.0000  , df_denom=230, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=30.9515 , p=0.0000  , df_denom=227, df_num=3
ssr based chi2 test:   chi2=95.7178 , p=0.0000  , df=3
likelihood ratio test: chi2=80.2424 , p=0.0000  , df=3
parameter F test:         F=30.9515 , p=0.0000  , df_denom=227, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=23.1530 , p=0.0000  

{1: ({'ssr_ftest': (20.378796378140404, 1.0087408823705415e-05, 233.0, 1),
   'ssr_chi2test': (20.641184314339636, 5.539164668882894e-06, 1),
   'lrtest': (19.787923702600494, 8.652764843610598e-06, 1),
   'params_ftest': (20.378796378143985, 1.0087408823688068e-05, 233.0, 1.0)},
   array([[0., 1., 0.]])]),
 2: ({'ssr_ftest': (41.19872338386215, 5.097314549456506e-16, 230.0, 2),
   'ssr_chi2test': (84.18869561050093, 5.231871019613086e-19, 2),
   'lrtest': (71.95627997182231, 2.3707858962998105e-16, 2),
   'params_ftest': (41.19872338385983, 5.097314549465169e-16, 230.0, 2.0)},
   array([[0., 0., 1., 0., 0.],
          [0., 0., 0., 1., 0.]])]),
 3: ({'ssr_ftest': (30.951487783025637, 8.207467068675482e-17, 227.0, 3),
   'ssr_chi2test': (95.71781684442287, 1.2943293538572461e-20, 3),
   'lrtest': (80.24236104091455, 2.7230095517890037e-17, 3),
   'params_ftest': (30.95148778302604, 8.207467068672012e-17, 227.0, 3.0)},
   array([[0., 0., 0., 1., 0., 0., 0.],
          [0., 0., 0., 0., 1.

### All

In [66]:
count_codes = [['AUDUSD.a','AU'], ['USDCAD.a', 'CA'], ['USDCHF.a','CH'], 
               ['GBPUSD.a', 'GB'], ['USDJPY.a', 'JP'], ['NZDUSD.a', 'NZ'], 
               ['USDSG.a', 'SG'], ['EURUSD.a', 'XM']]

for country in count_codes:
    print(f'D.N.N.{country[1]}')

D.N.N.AU
D.N.N.CA
D.N.N.CH
D.N.N.GB
D.N.N.JP
D.N.N.NZ
D.N.N.SG
D.N.N.XM


In [115]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd

combined_dfs = {}

# Define the base URL for the BIS Stats API
base_url = 'https://stats.bis.org/api/v1'

count_codes = [['AUDUSD.a','AU'], ['USDCAD.a', 'CA'],
               ['USDCHF.a','CH'], ['GBPUSD.a', 'GB'],
               ['USDJPY.a', 'JP'], ['NZDUSD.a', 'NZ'], 
               ['EURUSD.a', 'XM']]

for country in count_codes:

    # Define the endpoint parameters
    flow = 'BIS,WS_EER_D,1.0'  # Example: Version 1.0 of the WS_EER_M domain, maintained by the BIS
    key = f'D.N.N.{country[1]}'
    start_period = '2000'  # Example: Start year 2000
    end_period = '2023'  # Example: End year 2020
    detail = 'full'  # Example: All data and documentation

    # Construct the endpoint URL
    endpoint_url = f'{base_url}/data/{flow}/{key}/all'

    # Define the query parameters
    query_params = {
        'startPeriod': start_period,
        'endPeriod': end_period,
        'detail': detail
    }

    # Make the GET request
    response = requests.get(endpoint_url, params=query_params)

    # Check for a successful response
    if response.status_code == 200:
        # Assign the text of the response to xml_data
        xml_data = response.text

        # Parse the XML data
        root = ET.fromstring(xml_data)

        # Initialize empty lists to store the data
        time_periods = []
        obs_values = []

        # Iterate through the XML and extract the desired information
        for obs in root.findall(".//Obs"):
            time_period = obs.get('TIME_PERIOD')
            obs_value = obs.get('OBS_VALUE')
            time_periods.append(time_period)
            obs_values.append(obs_value)

        # Create a DataFrame
        df = pd.DataFrame({
            'Time_Period': time_periods,
            'OBS_Value': obs_values
        })

        df['OBS_Value'] = df['OBS_Value'].replace('NaN', np.nan)
        # Drop rows with NaN values
        df.dropna(subset=['OBS_Value'], inplace=True)
        df['OBS_Value'] = df['OBS_Value'].astype(float)
        df['Time_Period'] = pd.to_datetime(df['Time_Period'])
        
        df = df.set_index('Time_Period')
        print(f"Getting {country[0]}'s rates")
        rates = get_rates(country[0], mt5.TIMEFRAME_D1, 500)
        
        combined = pd.concat([df[-len(rates):], rates['close']], join = 'outer', axis = 1)
        
        combined_dfs[country[1]] = combined.dropna()

    else:
        print(f'Failed to retrieve data: {response.status_code}')

Getting AUDUSD.a's rates
Getting USDCAD.a's rates
Getting USDCHF.a's rates
Getting GBPUSD.a's rates
Getting USDJPY.a's rates
Getting NZDUSD.a's rates
Getting EURUSD.a's rates


In [116]:
from statsmodels.tsa.stattools import grangercausalitytests

import contextlib
import io
import statsmodels.api as sm

In [119]:
granger_dict = {}  # Initialize an empty dictionary

for name, df in combined_dfs.items():

    print(f"---------------------------------Iterating through {name}---------------------------------")
    
    f = io.StringIO()
    with contextlib.redirect_stdout(f):
        grang_test = grangercausalitytests(df[['OBS_Value', 'close']], 10)
    
    # grang_test = grangercausalitytests(df[['OBS_Value', 'close']], 10)
    
    # Initialize an empty list to store significant lags for the current column
    significant_lags = []
    for i in range(1, 10):
        score = 0

        for test in grang_test[i][0]:
            p_val = grang_test[i][0][test][1]
            if p_val < 0.05:
                score += 1 
        if score == 4:
            print(f"Lag {i} is significant")
            # Append the significant lag to the list
            significant_lags.append(i)

    if significant_lags:
        granger_dict[name] = significant_lags


---------------------------------Iterating through AU---------------------------------
Lag 1 is significant
Lag 2 is significant
Lag 3 is significant
Lag 4 is significant
Lag 5 is significant
Lag 6 is significant
Lag 7 is significant
Lag 8 is significant
Lag 9 is significant
---------------------------------Iterating through CA---------------------------------
Lag 1 is significant
Lag 2 is significant
Lag 3 is significant
Lag 4 is significant
Lag 5 is significant
Lag 6 is significant
Lag 7 is significant
Lag 8 is significant
Lag 9 is significant
---------------------------------Iterating through CH---------------------------------
Lag 2 is significant
Lag 3 is significant
Lag 4 is significant
Lag 5 is significant
Lag 6 is significant
Lag 7 is significant
Lag 8 is significant
Lag 9 is significant
---------------------------------Iterating through GB---------------------------------
Lag 1 is significant
Lag 2 is significant
Lag 3 is significant
Lag 4 is significant
Lag 5 is significant
L

In [120]:
granger_dict

{'AU': [1, 2, 3, 4, 5, 6, 7, 8, 9],
 'CA': [1, 2, 3, 4, 5, 6, 7, 8, 9],
 'CH': [2, 3, 4, 5, 6, 7, 8, 9],
 'GB': [1, 2, 3, 4, 5, 6, 7, 8, 9],
 'JP': [1, 2, 3, 4, 5, 6, 7, 8, 9],
 'NZ': [1, 2, 3, 4, 5, 6, 7, 8, 9],
 'XM': [2, 3, 4, 5, 6, 7, 8, 9]}